In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install line-bot-sdk
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import os
import pymc as pm
import numpy as np
import arviz as az
from linebot import LineBotApi
from linebot.models import TextSendMessage

# Google Drive上のJSON認証ファイルのパス
json_path = '/content/drive/MyDrive/.json'

# Google Sheets APIの認証設定
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
client = gspread.authorize(creds)

# スプレッドシートの読み込み
spreadsheet = client.open('weatherdata')  # Googleスプレッドシートの名前
worksheet = spreadsheet.sheet1  # シートの選択

# データ取得
data = worksheet.get_all_records()  # 全データを取得
data_array = np.array(data, dtype=object)

# 'time'列を除いたデータを処理
data_without_time = [
    {key: value for key, value in record.items() if key != 'time'} for record in data
]

# データをNumPy配列に変換
data_array_without_time = np.array(data_without_time)

def calculate_dry_score(data_without_time):
    alpha, beta, gamma, delta = 0.35, 0.25, 0.3, 3

    # 日照条件のスコア
    condition_map = {
        "Clear": 4.0,
        "Clouds": 1.0,
        "Rain": -2.0,
        "Snow": -3.0,
    }
    D = condition_map.get(data_without_time["condition"], 0.0)

    # スコア計算
    S = (alpha * data_without_time["temperature"] +
         beta * (100 - data_without_time["humidity"]) +
         gamma * data_without_time["wind_speed"] +
         delta * D)

    return S

# 各データにスコアを計算
scores = [calculate_dry_score(record) for record in data_array_without_time]

# 最新データの乾きやすさスコア
latest_data = data_without_time[-1]  # 最後のデータ
latest_score = calculate_dry_score(latest_data)

# 天気条件を数値に変換
def convert_condition_to_numeric(condition):
    if condition == "Clear":
        return 1
    elif condition == "Clouds":
        return 0
    else:
        return -1

# モデル用の入力データ準備
X_new_numeric = np.array([
    [
        record["temperature"],
        record["humidity"],
        record["wind_speed"],
        convert_condition_to_numeric(record["condition"])
    ] for record in data_array_without_time
])
y_new = np.array(scores)

# PyMCを用いたモデル定義とサンプリング
with pm.Model() as model:
    weights = pm.Normal("weights", mu=0, sigma=1, shape=3)  # 温度、湿度、風速用
    condition_effect = pm.Normal("condition_effect", mu=0, sigma=1, shape=4)  # 各条件に影響

    sigma = pm.HalfNormal("sigma", sigma=1)

    # 線形モデル
    X_effect = pm.math.dot(X_new_numeric[:, :-1], weights)  # 温度、湿度、風速
    condition_idx = X_new_numeric[:, -1].astype(int)  # condition 列を整数化
    mu = X_effect + condition_effect[condition_idx]

    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y_new)
    trace = pm.sample(2000, return_inferencedata=True)

# 結果確認
print(az.summary(trace, round_to=2))
az.plot_posterior(trace, var_names=["weights"])

# 最新データを反映したモデル更新
X_updated = np.vstack([
    X_new_numeric,
    [
        latest_data["temperature"],
        latest_data["humidity"],
        latest_data["wind_speed"],
        convert_condition_to_numeric(latest_data["condition"])
    ]
])
y_updated = np.append(y_new, latest_score)

# モデルを新しいデータで再学習
with pm.Model() as updated_model:
    weights = pm.Normal("weights", mu=0, sigma=1, shape=4)
    sigma = pm.HalfNormal("sigma", sigma=1)
    mu = pm.math.dot(X_updated, weights)
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y_updated)
    updated_trace = pm.sample(1000, return_inferencedata=True)

# LINEで通知
def get_drying_status(score):
    if score >= 18:
        return f"🌞 洗濯指数: {score:.2f} - 快晴で絶好の洗濯日和です！"
    elif score >= 10:
        return f"🌤 洗濯指数: {score:.2f} - 外干し可能ですが注意が必要です。"
    elif score >= 5:
        return f"☁️ 洗濯指数: {score:.2f} - 室内干しがおすすめです。"
    else:
        return f"🌧 洗濯指数: {score:.2f} - 洗濯物が乾きにくい天候です。"

# LINEボットAPIの設定
line_bot_api = LineBotApi('Your API')
user_ids = ['Your List']  # 送信先ユーザーIDリスト

# メッセージ作成
drying_status_message = get_drying_status(latest_score)

# メッセージ送信
for user_id in user_ids:
    line_bot_api.push_message(user_id, TextSendMessage(text=drying_status_message))